# Business Goal
TBD

# Data preparation

In [1]:
import pandas as pd
import numpy as np

### Read data sets

In [6]:
# Load the data from zip file
application_df = pd.read_csv('data/application_record.zip')
credit_df = pd.read_csv('data/credit_record.zip')

### Construct target label

In [15]:
credit_df.sample(5)

,ID,MONTHS_BALANCE,STATUS
957463,5137096,-26,0
69614,5004813,-3,X
891679,5121534,-17,C
2890,5001851,-5,C
882046,5121116,-35,C


***Content explanation***
* ID - Client number	　
* MONTHS_BALANCE - Record month	The month of the extracted data is the starting point, backwards, 0 is the current month, -1 is the previous month, and so on
* STATUS	Status	0: 1-29 days past due 1: 30-59 days past due 2: 60-89 days overdue 3: 90-119 days overdue 4: 120-149 days overdue 5: Overdue or bad debts, write-offs for more than 150 days C: paid off that month X: No loan for the month

In [16]:
# count unique ids in credit_df
cred_uids = credit_df['ID'].nunique()

# count unique ids in application_df
app_uids = application_df['ID'].nunique()

print(f'Unique IDs in credit_df: {cred_uids}')
print(f'Unique IDs in application_df: {app_uids}')

Unique IDs in credit_df: 45985
Unique IDs in application_df: 438510


Here we can see that credit records contain fewer unique IDs compared to application records. However, we need to filter credit records further.

To build a target label we will use the following strategy:
* "Bad" client - anyone who has records with 30 days overdue or more in the last two years
* "Good" client - anyone who has a credit record history for the last 2 years and no overdue over 30 days

Clients who don't match good or bad criteria will be ignored.

In [20]:
valid_credit_records = credit_df[credit_df['MONTHS_BALANCE'] >= -24]

print(f'Clients with a valid credit history: {valid_credit_records["ID"].nunique()}')

Clients with a valid credit history: 42596


In [21]:
# select all applications records that has id in valid_credit_records
valid_applications = application_df[application_df['ID'].isin(valid_credit_records['ID'])]

In [26]:
print(f'Amount of valid applications: {valid_applications["ID"].nunique()}')

Amount of valid applications: 33068


In [37]:
# add a column to valid_applications that indicates if the client is good or bad
valid_applications['STATUS'] = 1 # good
valid_applications.loc[valid_applications['ID'].isin(credit_df[credit_df['STATUS'].isin(['1','2','3','4', '5'])]['ID']), 'STATUS'] = 0 # bad

/var/folders/rz/d543tms9543f_v11vps9f2s00000gp/T/ipykernel_95348/333513147.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_applications['STATUS'] = 1 # good


In [38]:
valid_applications['STATUS'].value_counts()

STATUS
1    29057
0     4011
Name: count, dtype: int64

# Data Analysis